In [2]:
import numpy as np
import csv
from itertools import permutations

file = open('/home/johannes/Work/INF3490/oblig1/european_cities.csv', 'r+')
reader = csv.reader(file,delimiter=";")

distance = np.zeros((24,24))
cities = []

i = 0
for row in reader:
    #print row
    if i == 0:
        for j in range(len(row)):
            cities.append(row[j])
    else:
        for j in range(len(distance)):
            distance[i-1,j] = float(row[j])
    i += 1


In [3]:
# Exhaustive search

A = distance[0:3,0:3]

index_cities = np.zeros(len(cities))
for i in range(len(cities)):
    index_cities[i] = i
a = index_cities[0:3]
    
def dist(cities):
    tour = np.zeros(len(cities)+1)
    tour[0] = 0
    for i in range(len(cities)-1):
        tour[i+1] = A[cities[i],cities[i+1]]
    tour[-1] = A[cities[-1], cities[0]]
    return np.sum(tour)
    
temp = 10000000
for i in permutations(a):
    best = dist(i)
    if best < temp:
        temp = best
        
print best

#tour = np.zeros(len(A)+1)
  
#best_tour = tour
#best = 1000000
#for i in range(len(A)):
#    print i
#    tour[0] = A[i,i]
#    temp_A1 = np.delete(A,i,1)
#    for j in permutations(temp_A1[i]):
#        tour[1] = j[0]
#        x1,y1 = np.where(temp_A1 == j[0])
#        x2,y2 = np.where(temp_A1[:,y1] == 0) 
#        temp_A2 = np.delete(temp_A1,y1,1)
#        for k in permutations(temp_A2[int(x2)]):
#            tour[2] = k[0]
#            x1,y1 = np.where(temp_A2 == k[0])
#            x2,y2 = np.where(temp_A2[:,y1] == 0)
#            temp_A3 = np.delete(temp_A2,y1,1)
#            tour[3] = temp_A3[x2]
#            tour[4] = temp_A3[i]
#            temp = np.sum(tour)
#            if temp < best:
#                best = temp
#                best_tour = tour
#                start_city = cities[i]
                
#print best_tour, start_city, best
         

4024.99


/home/johannes/.local/lib/python2.7/site-packages/ipykernel/__main__.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/johannes/.local/lib/python2.7/site-packages/ipykernel/__main__.py:15: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
